### Importing libraries

In [50]:
%reset

In [51]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import json
import difflib
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

### Preprocessing

In [52]:
dataset=pd.read_csv("arabic_data.csv",on_bad_lines='skip')
dataset.columns=["title","contents","category","author_name"]
dataset.head(5),dataset.shape

(                                               title  \
 0                            بسم اللّه الرحمن الرحيم   
 1        بتمويل كويتي.. افتتاح مشروع للمياه في اليمن   
 2                                                  "   
 3  سمو نائب الأمير وولي العهد يهنئ الملك تشارلز ا...   
 4  سمو الأمير يهنئ تشارلز الثالث بمناسبة إعتلائه ...   
 
                                             contents  \
 0                                                      
 1  \\n                                    (كونا) ...   
 2                                            alwatan   
 3  (كونا) - بعث سمو نائب الأمير وولي العهد الشيخ ...   
 4  (كونا) - بعث سمو أمير البلاد الشيخ نواف الأحمد...   
 
                            category author_name  
 0  government performance follow up  local news  
 1                               NaN         NaN  
 2                        local news         NaN  
 3                           alqabas  local news  
 4                           alqabas  local news  ,
 (187

In [53]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18714 entries, 0 to 18713
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        18712 non-null  object
 1   contents     14419 non-null  object
 2   category     17706 non-null  object
 3   author_name  17704 non-null  object
dtypes: object(4)
memory usage: 584.9+ KB


In [54]:
dataset.dropna(axis=0,inplace=True)

In [55]:
dataset

,title,contents,category,author_name
0,بسم اللّه الرحمن الرحيم,,government performance follow up,local news
3,سمو نائب الأمير وولي العهد يهنئ الملك تشارلز ا...,(كونا) - بعث سمو نائب الأمير وولي العهد الشيخ ...,alqabas,local news
4,سمو الأمير يهنئ تشارلز الثالث بمناسبة إعتلائه ...,(كونا) - بعث سمو أمير البلاد الشيخ نواف الأحمد...,alqabas,local news
5,مستشفى الطب الطبيعي والتأهيل يحتفل باليوم العا...,احتفل مستشفى الطب الطبيعي والتأهيل باليوم العا...,alqabas,local news
6,«الهلال الأحمر» تبدأ حملة جمع تبرعات لمتضرري ا...,بدأت جمعية الهلال الأحمر اليوم السبت، حملة جمع...,alqabas,local news
...,...,...,...,...
18709,مستقبل نرعاه بالمعرفة,تستند مؤسسة الكويت للتقدم العلمي إلى ثلاثة محا...,\N,Sciences news
18710,لمزيد من المعلومات عن البرنامج التدريبي يرجى ا...,\N,\N,Sciences news
18711,تعلن مؤسسة الكويت للتقدم العلمي عن برنامج تدري...,\N,\N,Sciences news
18712,الأبحاث في مجال الوقود الأحفوري (التقليدي) وغي...,\N,\N,Sciences news


In [56]:
dataset.loc[dataset["category"]==r"\N"]["category"]

54       \N
81       \N
102      \N
111      \N
120      \N
         ..
18709    \N
18710    \N
18711    \N
18712    \N
18713    \N
Name: category, Length: 3064, dtype: object

In [57]:
empty_contents=dataset.loc[dataset["contents"]==r"\N"].index
empty_title=dataset.loc[dataset["title"]==r"\N"].index
print(empty_title)

Int64Index([], dtype='int64')


In [58]:
# removing all special characters 
import re
exp='[\u0627-\u064aA-Za-z]+' #re to get arabic and english , i had eliminated numbers and special characters because they dont have much influence 
dataset["contents"]=dataset['contents'].apply(lambda data : " ".join(re.findall(exp,data)))
dataset["category"]=dataset["category"].apply(lambda data : " ".join(re.findall(exp,data)))
dataset["title"]=dataset["title"].apply(lambda data : " ".join(re.findall(exp,data)))

In [59]:
dataset.loc[dataset["contents"]==r"\N"]

,title,contents,category,author_name


In [60]:
dataset.loc[dataset["contents"]==r"\N"]

,title,contents,category,author_name


In [61]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13897 entries, 0 to 18713
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        13897 non-null  object
 1   contents     13897 non-null  object
 2   category     13897 non-null  object
 3   author_name  13897 non-null  object
dtypes: object(4)
memory usage: 542.9+ KB


In [62]:
dataset.reset_index(inplace=True)

In [63]:
dataset.drop("index",axis=1,inplace=True)

In [64]:
dataset.head(5)

,title,contents,category,author_name
0,بسم الل ه الرحمن الرحيم,,government performance follow up,local news
1,سمو نا ب ال مير وولي العهد يهن الملك تشارلز ال...,كونا بعث سمو نا ب ال مير وولي العهد الشيخ مشعل...,alqabas,local news
2,سمو ال مير يهن تشارلز الثالث بمناسبة عتلا ه ال...,كونا بعث سمو مير البلاد الشيخ نواف ال حمد ببرق...,alqabas,local news
3,مستشفى الطب الطبيعي والت هيل يحتفل باليوم العا...,احتفل مستشفى الطب الطبيعي والت هيل باليوم العا...,alqabas,local news
4,الهلال ال حمر تبد حملة جمع تبرعات لمتضرري الصو...,بد ت جمعية الهلال ال حمر اليوم السبت حملة جمع ...,alqabas,local news


In [65]:
dataset.iloc[0]["contents"]

''

In [66]:
#still some empty strings are there --- remove

In [67]:
empty_str_contents=dataset.loc[dataset["contents"]==r""].index
empty_str_title=dataset.loc[dataset["title"]==r""].index
empty_str_category=dataset.loc[dataset["category"]==""].index

In [68]:
# empty_str_category, 
empty_str_contents,
# empty_str_title

(Int64Index([    0,    30,   111,   168,   193,   262,  3617,  3619,  3632,
              3795,  3815,  3828,  3903,  3971,  3976,  4011,  4050,  4051,
              4071,  4095,  4186,  4451,  4687,  4702,  4720,  4727,  4735,
              4737,  4740,  4742,  4769,  4773,  4781,  4793,  4801,  4810,
              4813,  4816,  4836,  4853,  6024,  6072,  6081,  7071,  7086,
              7138,  7148,  7171,  7266,  7286,  7326,  7458,  7462,  7480,
              7499,  7502,  7506,  7507,  8917,  9063,  9073,  9395,  9786,
              9792, 12744, 12993, 13198, 13201, 13205, 13599, 13677, 13678,
             13706, 13722, 13753, 13763, 13766],
            dtype='int64'),)

In [69]:
dataset.drop(list(empty_str_contents),axis=0,inplace=True)

In [70]:
empty_str_contents=dataset.loc[dataset["contents"]==r""].index
empty_str_contents

Int64Index([], dtype='int64')

In [71]:
dataset.reset_index(inplace=True)

In [72]:
dataset.drop("index",axis=1,inplace=True)

In [73]:
dataset.head(5)

,title,contents,category,author_name
0,سمو نا ب ال مير وولي العهد يهن الملك تشارلز ال...,كونا بعث سمو نا ب ال مير وولي العهد الشيخ مشعل...,alqabas,local news
1,سمو ال مير يهن تشارلز الثالث بمناسبة عتلا ه ال...,كونا بعث سمو مير البلاد الشيخ نواف ال حمد ببرق...,alqabas,local news
2,مستشفى الطب الطبيعي والت هيل يحتفل باليوم العا...,احتفل مستشفى الطب الطبيعي والت هيل باليوم العا...,alqabas,local news
3,الهلال ال حمر تبد حملة جمع تبرعات لمتضرري الصو...,بد ت جمعية الهلال ال حمر اليوم السبت حملة جمع ...,alqabas,local news
4,السفارة البريطانية رفع العلم من ظهر اليوم حتى ...,مي السكري علنت السفارة البريطانية لدى البلاد ن...,alqabas,local news


In [74]:
dataset.shape

(13820, 4)

In [75]:
dataset=dataset.iloc[1000:1040]

# keyword generation using YAKE
keywords are generated in combination of title and contents

In [76]:
dataset["keywords"]=dataset["title"]+" "+dataset["contents"]

C:\Users\rbw19\AppData\Local\Temp\ipykernel_13192\2632053667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["keywords"]=dataset["title"]+" "+dataset["contents"]


In [77]:
dataset.head(5)

,title,contents,category,author_name,keywords
1000,وزير ال وقاف دولة الكويت قيادة وحكومة وشعبا ته...,قال وزير ال وقاف والش ون ال سلامية الكويتي عيس...,alhrer news,local news,وزير ال وقاف دولة الكويت قيادة وحكومة وشعبا ته...
1001,السلمان ي عيد تشكيل مجلس الت ديب بجهاز حماية ا...,صدر وزير التجارة والصناعة د عبدالله السلمان قر...,alhrer news,local news,السلمان ي عيد تشكيل مجلس الت ديب بجهاز حماية ا...
1002,ثرى الكويت يحتضن جثامين شهيدا,تقدم وزير الداخلية وزير الدفاع بال نابة الشيخ ...,alhrer news,local news,ثرى الكويت يحتضن جثامين شهيدا تقدم وزير الداخل...
1003,الوزير العتيبي الكويت حريصة على صون حقوق الطفل...,كد وزير الكهربا والما والطاقة المتجددة وزير ال...,alhrer news,local news,الوزير العتيبي الكويت حريصة على صون حقوق الطفل...
1004,العنزي مشاركتنا بمعرض تونس للكتاب تعزز التعاون...,كد مس ول ثقافي كويتي همية مشاركة الكويت الرسمي...,alhrer news,local news,العنزي مشاركتنا بمعرض تونس للكتاب تعزز التعاون...


In [78]:
#running this cell may take 30 mins
from yake import KeywordExtractor
kw_extractor = KeywordExtractor(lan="ar", n=1, top=15) 
def keyword_extractor(data):
    keywords = kw_extractor.extract_keywords(text=data)
    keywords = " ".join([x for x, y in keywords])
    return keywords
dataset["keywords"]=dataset["keywords"].apply(lambda data:keyword_extractor(data))

C:\Users\rbw19\AppData\Local\Temp\ipykernel_13192\3561757148.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["keywords"]=dataset["keywords"].apply(lambda data:keyword_extractor(data))


In [79]:
dataset.head(5)

,title,contents,category,author_name,keywords
1000,وزير ال وقاف دولة الكويت قيادة وحكومة وشعبا ته...,قال وزير ال وقاف والش ون ال سلامية الكويتي عيس...,alhrer news,local news,القر الكريم وقاف الفا بلغ زين حصلت الكويت الكن...
1001,السلمان ي عيد تشكيل مجلس الت ديب بجهاز حماية ا...,صدر وزير التجارة والصناعة د عبدالله السلمان قر...,alhrer news,local news,حماية تشكيل المنافسة المجلس التجارة حكام القرا...
1002,ثرى الكويت يحتضن جثامين شهيدا,تقدم وزير الداخلية وزير الدفاع بال نابة الشيخ ...,alhrer news,local news,شهيدا وزير جثامين الكويت الشهدا الداخلية العلي...
1003,الوزير العتيبي الكويت حريصة على صون حقوق الطفل...,كد وزير الكهربا والما والطاقة المتجددة وزير ال...,alhrer news,local news,الكويت الطفل حقوق حرص كافة باليوم العالمي والع...
1004,العنزي مشاركتنا بمعرض تونس للكتاب تعزز التعاون...,كد مس ول ثقافي كويتي همية مشاركة الكويت الرسمي...,alhrer news,local news,المعرض الكويت الثقافية الكتاب العربية العنزي ل...


In [ ]:
dataset.to_csv("news_data.csv")

# TFIDF vectorization

In [33]:
combined_features=dataset.title+" "+dataset.contents+" "+dataset.category+" "+dataset.author_name+" "+dataset.keywords

In [34]:
combined_features[1]

'سمو ال مير يهن تشارلز الثالث بمناسبة عتلا ه العرش ملكا للمملكة المتحدة كونا بعث سمو مير البلاد الشيخ نواف ال حمد ببرقية تهن ة لى صاحب الجلالة الملك تشارلز الثالث ملك المملكة المتحدة alqabas local news تشارلز الثالث سمو مير المتحدة يهن بمناسبة عتلا العرش ملكا للمملكة كونا بعث البلاد الشيخ'

In [35]:
tfidf=TfidfVectorizer()
tfidf.fit(combined_features)

TfidfVectorizer()

In [ ]:
# save the tfidf model for further use

In [36]:
new_df = tfidf.transform(combined_features)

In [37]:
new_df

<100x4492 sparse matrix of type '<class 'numpy.float64'>'
	with 8414 stored elements in Compressed Sparse Row format>

In [42]:
tfidf.vocabulary_

{'سمو': 2386,
 'نا': 3566,
 'ال': 137,
 'مير': 3552,
 'وولي': 4335,
 'العهد': 838,
 'يهن': 4476,
 'الملك': 1173,
 'تشارلز': 1756,
 'الثالث': 362,
 'كونا': 2884,
 'بعث': 1551,
 'الشيخ': 694,
 'مشعل': 3391,
 'حمد': 2081,
 'ببرقية': 1454,
 'تهن': 1891,
 'لى': 3206,
 'صاحب': 2485,
 'الجلالة': 393,
 'ملك': 3465,
 'alqabas': 5,
 'local': 18,
 'news': 22,
 'بمناسبة': 1610,
 'عتلا': 2597,
 'العرش': 802,
 'ملكا': 3466,
 'للمملكة': 3150,
 'المتحدة': 988,
 'البلاد': 265,
 'نواف': 3658,
 'المملكة': 1175,
 'مستشفى': 3366,
 'الطب': 750,
 'الطبيعي': 753,
 'والت': 3763,
 'هيل': 3696,
 'يحتفل': 4377,
 'باليوم': 1449,
 'العالمي': 773,
 'للعلاج': 3114,
 'احتفل': 53,
 'والذي': 3819,
 'يوافق': 4480,
 'يوم': 4488,
 'الثامن': 364,
 'من': 3484,
 'سبتمبر': 2319,
 'كل': 2868,
 'عام': 2577,
 'وكشف': 4228,
 'مدير': 3321,
 'الهلال': 1281,
 'حمر': 2082,
 'تبد': 1658,
 'حملة': 2085,
 'جمع': 1975,
 'تبرعات': 1660,
 'لمتضرري': 3167,
 'الصومال': 735,
 'والسودان': 3833,
 'وباكستان': 3947,
 'بد': 1493,
 'جمعية': 1977,
 '

In [43]:
len(tfidf.vocabulary_)

4492

In [38]:
temp=pd.DataFrame.sparse.from_spmatrix(new_df)

,0,1,2,3,4,5,6,7,8,9,...,4482,4483,4484,4485,4486,4487,4488,4489,4490,4491
0,0.0,0.0,0.000000,0.0,0.000000,0.068662,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.068116,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.048165,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.068701,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.050477,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.056797,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.087429,...,0.0,0.0,0.0,0.0,0.101759,0.0,0.000000,0.0,0.0,0.0
96,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
97,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
98,0.0,0.0,0.099227,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


<img src="https://cs.carleton.edu/cs_comps/0910/netflixprize/final_results/knn/img/knn/cos.png">

In [ ]:
similarity_table=cosine_similarity(new_df) #generates the similarity table between all the news #return type numpy array

In [ ]:
similarity_table.shape #gives 13820 x 13820 -- that is similarity between the vectors of 13820 news

(13820, 13820)

In [ ]:
new_df=pd.DataFrame.sparse.from_spmatrix(new_df)

# Testing the model

In [40]:
def similarity_predictor(data):
    # closly_matched=difflib.get_close_matches(data,dataset.title.to_list()) #this function has been used to match input movie name with all similar movie names we can do that with a for but handling case sensitivity and spacing mistakes make our task tuff that why we use this library 
    # print(closly_matched) #returns a list of close matches
    # confirmation=int(input("enter the index value for movie"))
    # exact_match=closly_matched[confirmation]
    # ##finding the vector for this movie from our initial dataset
    index_num = dataset.loc[dataset.title==data].index[0]
    print(index_num)
    print("vector of our movie",new_df.iloc[index_num])
    similarity_scores=list(enumerate(similarity_table[index_num])) # finding the index value in similarity table
    # print(similarity_scores)

    res = sorted(similarity_scores, key = lambda x: x[1], reverse = True)[:5]
    print(res)
    print("Predictions are")
    for i in res:
        print("prediction",dataset.iloc[i[0]]["title"])
    
title=dataset.iloc[8].title 
content=dataset.iloc[8].contents
similarity_predictor(title)

8
vector of our movie 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
108465    0.0
108466    0.0
108467    0.0
108468    0.0
108469    0.0
Name: 8, Length: 108470, dtype: Sparse[float64, 0]
[(8, 1.0000000000000002), (3462, 0.6544164446457322), (3666, 0.5847996069766551), (101, 0.5770757419137155), (386, 0.566502605113999)]
Predictions are
prediction الوزير الموسى زيادة مواقع نقاط شواحن المركبات الكهربا ية بالكويت
prediction الموسى الانتها من عداد لا حة الشواحن لوضع استراتيجيات منظومة توفير الشواحن للسيارات الكهربا ية بالكويت
prediction الموسى الانتها من عداد لا حة الشواحن للسيارات الكه
prediction وزير الكهربا الانتها من عداد لا حة شواحن السيارات الكهربا ية في الكويت
prediction الموسى الانتها من عداد لا حة الشواحن لوضع استراتيجيات منظومة توفير الشواحن للسيارات الكهربا ية بالكويت


In [41]:
title=dataset.iloc[8].title 
title

'الوزير الموسى زيادة مواقع نقاط شواحن المركبات الكهربا ية بالكويت'